In [1]:
import os
import sys
import contextlib
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from argparse import Namespace

sns.set(context="talk", palette="husl", color_codes=True)


class DummyFile(object):
    def write(self, x): pass

@contextlib.contextmanager
def nostdout():
    save_stdout = sys.stdout
    sys.stdout = DummyFile()
    yield
    sys.stdout = save_stdout

In [2]:
SIM_NAME = "DIG_bright"
EXP_TIME = 900

SIM_PATH = f"_data-sim/{SIM_NAME}/outputs"

SIMPLE_SIM_NAME = f"{SIM_NAME}_linear_full_{EXP_TIME}_flux.fits"

current_sim = SIMPLE_SIM_NAME
out_fl_name = os.path.join(SIM_PATH, current_sim.replace(".fits", ".fl.rss.fits"))
out_ef_name = os.path.join(SIM_PATH, current_sim.replace(".fits", ".ef.rss.fits"))

In [3]:
args = Namespace()
args.spec_file = out_fl_name
args.error_file = out_ef_name

args.single_gas_fit = True
args.ignore_gas = False

args.emission_lines_file = "../../_fitting-data/_configs/MaNGA/emission_lines_long_list.MaNGA"
args.output_path = "_data/out"

args.sigma_gas = 3

args.config_file = None

args.rsp_file = "../../_fitting-data/_basis_mastar_v3.1.1/stellar-basis-spectra-100.fits.gz"
args.rsp_nl_file = "../../_fitting-data/_basis_mastar_v3.1.1/stellar-basis-spectra-10.fits.gz"

args.mask_file = None

args.sigma_inst = 0.001
args.flux_scale = -np.inf, np.inf

args.w_range = 3650.0, 9800.0
args.w_range_nl = 3800, 4700

args.redshift = 0, 0, 0, 0.1
args.sigma = 0, 0, 0, 350
args.AV = 0, 0, 0, 1.5

args.plot = 0
args.ratio = True
args.y_ratio = None
args.fit_sigma_rnd = True

args.label = current_sim.replace(".fits", "")

out_file_elines = os.path.join(args.output_path, f"elines_{args.label}")
out_file_single = os.path.join(args.output_path, f"single_{args.label}")
out_file_coeffs = os.path.join(args.output_path, f"coeffs_{args.label}")
out_file_fit = os.path.join(args.output_path, f"output.{args.label}.fits.gz")
out_file_ps = os.path.join(args.output_path, args.label)

In [4]:
from pyFIT3D.common.auto_ssp_tools import dump_rss_output
from pyFIT3D.common.auto_ssp_tools import load_rss
from lvmdap._cmdline.dap import auto_rsp_elines_rnd


wl__w, rss_flux, rss_eflux = load_rss(spec_file=args.spec_file, error_file=args.error_file)

sigma_seq = []
input_delta_sigma = args.sigma[1]
input_min_sigma = args.sigma[2]
input_max_sigma = args.sigma[3]
model_spectra = []

y_ratio = None
ns = rss_flux.shape[0]

for i, (f__w, ef__w) in tqdm(enumerate(zip(rss_flux, rss_eflux)), total=ns, desc="fitting spectra", ascii=True, unit="spectrum"):
    with nostdout():
        _, SPS = auto_rsp_elines_rnd(
            wl__w, f__w, ef__w, ssp_file=args.rsp_file, ssp_nl_fit_file=args.rsp_nl_file,
            config_file=args.config_file,
            w_min=args.w_range[0], w_max=args.w_range[1], nl_w_min=args.w_range_nl[0],
            nl_w_max=args.w_range_nl[1], mask_list=args.mask_file,
            min=args.flux_scale[0], max=args.flux_scale[1], elines_mask_file=args.emission_lines_file,
            fit_gas=not args.ignore_gas, refine_gas=not args.single_gas_fit, sigma_gas=args.sigma_gas,
            input_redshift=args.redshift[0], delta_redshift=args.redshift[1], min_redshift=args.redshift[2], max_redshift=args.redshift[3],
            input_sigma=args.sigma[0], delta_sigma=args.sigma[1], min_sigma=args.sigma[2], max_sigma=args.sigma[3],
            input_AV=args.AV[0], delta_AV=args.AV[1], min_AV=args.AV[2], max_AV=args.AV[3], y_ratio=y_ratio,
            sigma_inst=args.sigma_inst, spaxel_id=f"{args.label}_{i}", out_path=args.output_path, plot=args.plot
        )
    y_ratio = SPS.ratio_master

    SPS.output_gas_emission(filename=out_file_elines, spec_id=i)
    SPS.output_coeffs_MC(filename=out_file_coeffs, write_header=i==0)
    SPS.output(filename=out_file_ps, write_header=i==0, block_plot=False)

    obs = SPS.spectra["orig_flux_ratio"]
    mod = SPS.spectra["model_joint"]

    fig, ax = plt.subplots(figsize=(25,5))
    plt.plot(wl__w, obs, "0.7", lw=5)
    plt.plot(wl__w, mod)
    plt.xlim(6500,6800)
    plt.savefig(os.path.join(args.output_path, f"{args.label}_{i}.png"), bbox_inches="tight")
    plt.close(fig)

    model_spectra.append(SPS.output_spectra_list)

model_spectra = np.array(model_spectra).transpose(1, 0, 2)

dump_rss_output(out_file_fit=out_file_fit, wavelength=wl__w, model_spectra=model_spectra)

fitting spectra:   0%|          | 0/1801 [00:00<?, ?spectrum/s]/home/mejia/.cache/pypoetry/virtualenvs/lvm-dap-fAIgvtcU-py3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mejia/.cache/pypoetry/virtualenvs/lvm-dap-fAIgvtcU-py3.8/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


3697-5176 wavelength range: Saving 22 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
5191-6435 wavelength range: Saving 25 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
6730-8236 wavelength range: Saving 23 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
8446-9399 wavelength range: Saving 7 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
-------------------------------------------------------------------------------------------------------------
|  ID  |  TEFF   |  LOGG   |  META   | ALPHAM  | COEFF  | Min.Coeff | log(M/L) |  AV  | N.Coeff | Err.Coeff |
-------------------------------------------------------------------------------------------------------------
| 0028 |  3.8476 |  4.4402 |  0.0350 | -0.0250 | 0.3820 |    0.3820 |  13.1113 | 0.00 |  0.3820 |    0.0000 | 0.3820 | 0.0000
| 0029 |  3.8901 |  4.7868 | -0.1984 | -0.0742 | 0.5802 |    0.5802 |  14.4435 | 0.00 |  0.5802 |    0.0000 | 0.5802 

fitting spectra:   0%|          | 1/1801 [04:07<123:53:27, 247.78s/spectrum]/home/mejia/.cache/pypoetry/virtualenvs/lvm-dap-fAIgvtcU-py3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mejia/.cache/pypoetry/virtualenvs/lvm-dap-fAIgvtcU-py3.8/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting spectra:   0%|          | 2/1801 [08:41<131:33:07, 263.25s/spectrum]

3686-5176 wavelength range: Saving 53 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
5191-6716 wavelength range: Saving 22 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
6730-7452 wavelength range: Saving 8 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
8300-9531 wavelength range: Saving 20 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
-------------------------------------------------------------------------------------------------------------
|  ID  |  TEFF   |  LOGG   |  META   | ALPHAM  | COEFF  | Min.Coeff | log(M/L) |  AV  | N.Coeff | Err.Coeff |
-------------------------------------------------------------------------------------------------------------
| 0035 |  3.9220 |  2.9127 |  0.4975 |  0.2669 | 0.0795 |    0.0797 |  12.0656 | 0.00 |  0.0795 |    0.0006 | 0.0801 | 0.0006
| 0040 |  3.7782 |  4.3005 | -0.5922 |  0.0448 | 0.0634 |    0.0629 |  13.1688 | 0.00 |  0.0634 |    0.0019 | 0.0621 

/home/mejia/.cache/pypoetry/virtualenvs/lvm-dap-fAIgvtcU-py3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mejia/.cache/pypoetry/virtualenvs/lvm-dap-fAIgvtcU-py3.8/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
fitting spectra:   0%|          | 3/1801 [13:16<133:58:37, 268.25s/spectrum]

...

3697-5176 wavelength range: Saving 54 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
5200-6716 wavelength range: Saving 22 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
6730-6730 wavelength range: Saving 2 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
8300-9682 wavelength range: Saving 28 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
-------------------------------------------------------------------------------------------------------------
|  ID  |  TEFF   |  LOGG   |  META   | ALPHAM  | COEFF  | Min.Coeff | log(M/L) |  AV  | N.Coeff | Err.Coeff |
-------------------------------------------------------------------------------------------------------------
| 0044 |  3.6175 |  2.6876 | -0.8062 |  0.3869 | 0.0006 |    0.0006 |  12.9288 | 0.00 |  0.0006 |    0.0000 | 0.0006 | 0.0000
| 0059 |  3.6092 |  1.3838 |  0.1434 |  0.0343 | 0.0640 |    0.0625 |  13.0951 | 0.00 |  0.0640 |    0.0046 | 0.0597 

fitting spectra:   4%|3         | 68/1801 [4:46:36<108:19:02, 225.01s/spectrum]

3686-5072 wavelength range: Saving 38 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
5270-6716 wavelength range: Saving 7 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
6730-6730 wavelength range: Saving 2 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
8308-9682 wavelength range: Saving 24 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
-------------------------------------------------------------------------------------------------------------
|  ID  |  TEFF   |  LOGG   |  META   | ALPHAM  | COEFF  | Min.Coeff | log(M/L) |  AV  | N.Coeff | Err.Coeff |
-------------------------------------------------------------------------------------------------------------
| 0000 |  3.9970 |  2.5946 | -1.0825 |  0.1597 | 0.0003 |    0.0003 |  14.0274 | 0.00 |  0.0003 |    0.0000 | 0.0003 | 0.0000
| 0006 |  3.9397 |  2.8905 | -2.3637 |  0.0845 | 0.0022 |    0.0021 |  11.0819 | 0.00 |  0.0022 |    0.0003 | 0.0020 |

fitting spectra:   4%|3         | 69/1801 [4:50:31<109:37:50, 227.87s/spectrum]

3686-5184 wavelength range: Saving 35 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
5270-6716 wavelength range: Saving 8 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
6730-7637 wavelength range: Saving 4 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
8308-9545 wavelength range: Saving 23 EML systems to file _data/out/elines_DIG_bright_linear_full_900_flux
-------------------------------------------------------------------------------------------------------------
|  ID  |  TEFF   |  LOGG   |  META   | ALPHAM  | COEFF  | Min.Coeff | log(M/L) |  AV  | N.Coeff | Err.Coeff |
-------------------------------------------------------------------------------------------------------------
| 0005 |  3.9837 |  3.4683 | -2.0871 |  0.2826 | 0.0199 |    0.0194 |  11.7220 | 0.00 |  0.0199 |    0.0015 | 0.0185 | 0.0014
| 0008 |  4.0046 |  3.0858 | -2.6604 |  0.0534 | 0.0003 |    0.0003 |  11.9797 | 0.00 |  0.0003 |    0.0001 | 0.0002 |